
Задача: запустить модель LDA и Gibbs Sampling с числов тегов 20. Вывести топ-10 слов по каждому тегу. Соотнести полученные теги с тегами из датасета. Добейтесь того, чтобы хотя бы несколько тем были явно интерпретируемы, например, как в примерах ниже.

Примеры топ-10 слов из некотрых тегов, которые получаются после применения LDA:
* ['god', 'jesus', 'believe', 'life', 'bible', 'christian', 'world', 'church', 'word', 'people'] - эта группа явно соотносится с soc.religion.christian
* ['drive', 'card', 'hard', 'bit', 'disk', 'scsi', 'memory', 'speed', 'mac', 'video'] - эту группу можно соотнести с темами 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware'
* ['game',	'games',	'hockey',	'league',	'play',	'players',	'season',	'team',	'teams',	'win'] - тема rec.sport.hockey

Советы:
* модель будет сходится лучше и быстрее, если уменьшить размер словаря за счет отсеивания общеупотребительных слов и редких слов. Управлять размером словаря можно с помощью параметров min_df (отсеивает слова по минимальной частоте встречаемости) и max_df (отсеивает слова по максимальной частоте встречаемости) в CountVectorizer.
* параметры $\alpha$, $\beta$ можно, для начала, положить единицами
* после 100 итераций можно ожидать хорошего распределения по темам. Если этого не происходит и в темах мешинина - проверяйте код и оптимизируйте словарь
* на примере третьей темы видно, что у нас встречаются разные формы одного и того же слова. С помощью процедур stemming и lemmatization можно привести слова к общей форме и объединить близкие по значению

In [20]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

In [37]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(lowercase=True, stop_words="english", analyzer='word', binary=True, min_df = 0.0001, max_df = 0.001)
vectorizer.fit(newsgroups_train.data)

CountVectorizer(binary=True, max_df=0.001, min_df=0.0001, stop_words='english')

In [38]:
X_train = vectorizer.fit_transform(newsgroups_train.data)
X_train.shape

(11314, 30063)

In [39]:
u = X_train.toarray()

In [40]:
print(vectorizer.vocabulary_.get(25717))

None


С этого места можно полностью собрать алгоритм моделирования плотности $p(\textbf{z}| \textbf{w}, \alpha, \beta)$. Введем  обозначения 
* $n_k$ - количество слов, отнесенных к тегу $k$, по всем документам
* $n_{k, w}$ - количество раз, когда слово $w$ вошло в тег $k$
* $n_{d, k}$ - количество вхождений тега $k$ в документ $d$
* $W$ - общее количество слов в корпусе документов
* $\beta_{sum} = \sum\limits_{w=1}^{N}\beta_w$

* заведем счетчики $n_{k, w}$, $n_{d, k}$, $n_k$, заполненные нулями

In [41]:
tags = 20
n_k = np.zeros(tags)
n_k_w = np.zeros(tags * X_train.shape[1]).reshape(tags,X_train.shape[1])
n_d_k = np.zeros(tags * X_train.shape[0]).reshape(X_train.shape[0],tags)

* случайным образом расставим теги словам, обновим счетчики $n_{k, w}$, $n_{d, k}$, $n_k$


In [42]:
dc, word = X_train.nonzero()
z = np.random.choice(tags, len(dc))

for i, j, k in zip(dc, word, z):
    n_k[k] += 1
    n_k_w[k, j] += 1
    n_d_k[i, k] += 1

* пока не сойдемся к стационарному режиму:
  * для каждого $i$ от 1 до $W$:
      * $n_{d_i, z_i} \mathrel{-}= 1$, $n_{z_i, w_i} \mathrel{-}= 1$, $n_{z_i} \mathrel{-}= 1$
      * для каждого $k$ от 1 до $K$:
        * вычисляем $p_k = (n_{d, k} + \alpha_k) \frac{n_{k, w_i} + \beta_{w_i}}{n_k + \beta_{sum}}$
      * сэмплим новый $z_i$ из полученного распределения $(p_1, ..., p_K)$
      * $n_{d_i, z_i} \mathrel{+}= 1$, $n_{z_i, w_i} \mathrel{+}= 1$, $n_{z_i} \mathrel{+}= 1$

In [43]:
def LDA(n_d_k, n_k_w, n_k, z, dc, word, tags, alpha, betta, it):    
    for i in range(it):
        for j in range(len(dc)):
            cword = word[j]
            cdc = dc[j]
            corder = z[j]

            n_k[corder] -= 1
            n_k_w[corder, cword] -= 1
            n_d_k[cdc, corder] -= 1

            p = (n_d_k[cdc, :] + alpha) * (n_k_w[:, cword] + betta[cword]) / (n_k + betta.sum())
            
            z[j] = np.random.choice(np.arange(tags), p = p / p.sum())
            
            n_k[corder] += 1
            n_k_w[corder, cword] += 1
            n_d_k[cdc, corder] += 1
    return z, n_k_w, n_d_k, n_k

* зададим априорные $\beta$, $\alpha$

In [44]:
alpha = np.ones(20)
betta = np.ones(X_train.shape[1])

Основная часть алгоритма:

In [45]:
z ,n_k_w, n_d_k, n_k = LDA(n_d_k, n_k_w, n_k, z, dc, word, 20, alpha, betta, 100)

Печатаем топ значений

In [47]:
result = np.argsort(n_k_w, axis=1)[:, -10:]
for i in range(20):
    matrix = np.zeros((1, X_train.shape[1]))
    for j in result[i]:
        matrix[0, j] = 1
    print('\nTAG  {}:   {}'.format(i+1, '\t'.join(vectorizer.inverse_transform(matrix)[0])))


TAG  1:   batman	compressor	parliamentary	quartz	respecting	slam	spacelink	supervisor	w0	x7

TAG  2:   1922	1949	aspirations	ccwf	forbids	kovalev	lag	neurons	populated	spray

TAG  3:   32k	attackers	b8e	catalogs	coin	fla	iifx	irresponsible	persists	unicorns

TAG  4:   boulevard	cats	ceased	dating	derivative	emphasizes	encounters	raids	spies	superstar

TAG  5:   0x	6j	circumference	discrepancy	excommunicated	intolerance	qgq	reichel	simtel20	worded

TAG  6:   0w	2tg	believable	calmly	enclosed	equalizer	financially	grandmother	ingredients	modifying

TAG  7:   airplanes	crooks	decree	dutch	fahrenheit	gainey	graduation	massage	payne	wingers

TAG  8:   1980s	beastmaster	fascism	heavier	hiking	melt	phigs	prescription	tennessee	und

TAG  9:   authoritarian	bleeding	bureaucracy	cancel	editions	ips	monkey	rpw	rub	surgeons

TAG  10:   955	citizenry	devotion	employers	induction	noring	popping	securing	teens	wireframe

TAG  11:   9m	drawbacks	entryway	fw	g6	internals	minimally	psi	y4	yah

TAG  12: